In [10]:
from typing import overload, TypeVarTuple


Cs = TypeVarTuple("Cs")


# @overload
# def f(c1: int) -> tuple[int, tuple[int]]:
#     ...

# @overload
# def f(c1: int, c2: int) -> tuple[int, tuple[int, int]]:
#     ...

def f(*cs: tuple[*Cs]) -> tuple[int, tuple[*Cs]]:
    return len(cs), cs


In [91]:
from dataclasses import dataclass
from typing import Any, Generic, Iterable, NewType, Optional, TypeVar, TypeVarTuple


Entity = NewType("Entity", int)

Cs = TypeVarTuple("Cs")


class Query(Generic[*Cs]):
    def __iter__(self) -> Iterable[tuple[Entity, tuple[*Cs]]]:
        ...


R = TypeVar("R")


class Resource(Generic[R]):
    def __init__(self, resource_type: type[R]):
        self.resource_type = resource_type


class Commands:
    def spawn_entity(self, *components: Any) -> Entity:
        ...

    def add_component(self, entity: Entity, component: Any) -> None:
        ...

    def remove_component(self, entity: Entity, component: type) -> None:
        ...


@dataclass
class Vec2:
    x: float = 0
    y: float = 0


class Position(Vec2):
    pass


class Velocity(Vec2):
    pass


class Name(str):
    pass


@dataclass
class PhysicsConstants:
    gravity: float = 9.81
    air_drag: float = 0.1

t: tuple[ int, str ] = (1, "2")

def update_position(
    cmds: Commands,
    # c: Resource[PhysicsConstants],
    c2: Resource[Optional[PhysicsConstants]],
    entities: Query[Optional[Name], Position, Velocity],
) -> None:
    ...

In [92]:
import inspect
from typing import Callable, Literal, Union


@dataclass
class ParameterCommands:
    name: str


@dataclass
class ParameterResource:
    name: str
    component: type
    optional: bool = False


@dataclass
class QueryArg:
    component: type
    optional: bool = False


@dataclass
class ParameterQuery:
    name: str
    args: list[QueryArg]


Parameter = ParameterCommands | ParameterResource | ParameterQuery


def unwrap_optional(t) -> tuple[type, bool]:
    optional = str(t).startswith("typing.Optional")
    if optional:
        return t.__args__[0], True

    return t, False


def inspect_system(system: Callable[..., None]):
    out: list[Parameter] = []

    signature = inspect.signature(system)
    for name, parameter in signature.parameters.items():
        annotation = parameter.annotation

        if annotation is Commands:
            out.append(ParameterCommands(name))
        elif annotation.__origin__ is Resource:
            component, optional = unwrap_optional(annotation.__args__[0])
            out.append(ParameterResource(name, component, optional))
        elif annotation.__origin__ is Query:
            args = []
            for arg in annotation.__args__:
                component, optional = unwrap_optional(arg)
                args.append(QueryArg(component, optional))
            out.append(ParameterQuery(name, args))
        else:
            raise ValueError(f"Invalid typing for parameter {parameter.name} in System: {annotation}")

    return out


inspect_system(update_position)

[ParameterCommands(name='cmds'),
 ParameterResource(name='c2', component=<class '__main__.PhysicsConstants'>, optional=True),
 ParameterQuery(name='entities', args=[QueryArg(component=<class '__main__.Name'>, optional=True), QueryArg(component=<class '__main__.Position'>, optional=False), QueryArg(component=<class '__main__.Velocity'>, optional=False)])]